In [2]:
from PIL import Image
import os
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont, ImageFilter


# Load the YOLO model
model_path = '../YOLOv8x_Symbols.pt'
model = YOLO(model_path)

# Directory containing images
source_dir = './sample/'
image_files = [os.path.join(source_dir, f) for f in os.listdir(source_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Run inference on all images
results = model(image_files)

# Directory to save annotated images
save_directory = './'
os.makedirs(save_directory, exist_ok=True)

def get_contrast_color(bg_color):
    # Calculate the luminance of the background color
    # using the formula for luminance under the sRGB Luma (Rec. 709)
    luminance = (0.299 * bg_color[0] + 0.587 * bg_color[1] + 0.114 * bg_color[2]) / 255
    # Return white if the background is dark; black if the background is light
    return (255, 255, 255) if luminance < 0.5 else (0, 0, 0)


# Function to generate unique colors for each class ID
def get_unique_color(tag):
    np.random.seed(tag)  # Seed with tag to get consistent color for the same tag
    return [int(x) for x in np.random.randint(0, 255, 3)]

for result in results:
    img_path = result.path
    image = Image.open(result.path).convert("RGB")
    # Draw predictions on the image
    draw = ImageDraw.Draw(image)

    if image is None:
        print(f"Failed to load image {img_path}")
        continue

    boxes = result.boxes.data
    confs = result.boxes.conf
    cls_ids = result.boxes.cls

    for box, conf, cls_id in zip(boxes, confs, cls_ids):
        x1, y1, x2, y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])
        conf = float(conf)
        cls_id = int(cls_id)
        label = result.names[cls_id]
        label_text = f'{label} {conf:.2f}'
        font_scale = 2
        color = get_unique_color(cls_id)
        text_color = get_contrast_color(color)

        # Draw bounding box
        draw.rectangle([( x1, y1), (x2, y2)], outline='red', width=3)

        font_size = 20  # Change this to the desired font size 
    
        # Draw label text in color matching the bounding box
        draw.text((x1, y1 - 40), label_text, fill='blue')

    save_path = os.path.join(save_directory, os.path.basename(img_path))
    image.save(save_path)



0: 992x704 4 braces, 1 coda, 5 clefGs, 5 clefFs, 98 noteheadBlackOnLines, 72 noteheadBlackInSpaces, 5 noteheadHalfOnLines, 2 noteheadHalfInSpaces, 2 accidentalNaturals, 13 accidentalSharps, 6 keySharps, 2 fermataAboves, 1 restHalf, 32 rest8ths, 46 beams, 6 staffs, 19.3ms
1: 992x704 6 clefGs, 2 clefFs, 16 timeSig4s, 34 noteheadBlackOnLines, 35 noteheadBlackInSpaces, 1 noteheadHalfOnLine, 3 flag8thDowns, 7 keyFlats, 6 restWholes, 3 restHalfs, 17 restQuarters, 2 rest8ths, 6 dynamicPs, 15 beams, 1 tie, 8 staffs, 19.3ms
Speed: 6.3ms preprocess, 19.3ms inference, 1.8ms postprocess per image at shape (1, 3, 992, 704)


In [4]:
print(result[0].boxes)

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([154.], device='cuda:0')
conf: tensor([0.9770], device='cuda:0')
data: tensor([[2.3257e+02, 2.0378e+03, 1.8643e+03, 2.1048e+03, 9.7699e-01, 1.5400e+02]], device='cuda:0')
id: None
is_track: False
orig_shape: (2772, 1960)
shape: torch.Size([1, 6])
xywh: tensor([[1048.4502, 2071.3003, 1631.7665,   66.9996]], device='cuda:0')
xywhn: tensor([[0.5349, 0.7472, 0.8325, 0.0242]], device='cuda:0')
xyxy: tensor([[ 232.5669, 2037.8004, 1864.3334, 2104.8000]], device='cuda:0')
xyxyn: tensor([[0.1187, 0.7351, 0.9512, 0.7593]], device='cuda:0')
